# Setup

In [1]:
import psycopg2
import pandas as pd
import os
import boto3
from botocore.exceptions import ClientError


In [2]:
# Hàm tạo kết nối tới PostgreSQL
def create_connection(db_name="token"):
    try:
        conn = psycopg2.connect(
            dbname=db_name,  
            user="HTP",         
            password="htp1782k1",  
            host="localhost",  
            port="5432"       
        )
        print(f"Kết nối đến PostgreSQL ({db_name}) thành công!")
        return conn
    except Exception as e:
        print(f"Không thể kết nối đến PostgreSQL: {e}")
        return None

In [3]:
# Cấu hình thông tin MinIO
minio_endpoint = 'http://localhost:9000' 
access_key = 'k8N26KNw26XudBhDIZXh'  
secret_key = 'DxuxIyTbIcJqjCFLB8FvuITeJjqWHvUm1ElHtGy2'  
bucket_name = 'rawdata'  

# Khởi tạo client MinIO
s3 = boto3.client('s3', endpoint_url=minio_endpoint,
                  aws_access_key_id=access_key,
                  aws_secret_access_key=secret_key)


# Tạo database

In [4]:
def create_database():
    try:
        conn = psycopg2.connect(
            dbname="postgres",  
            user="HTP",       
            password="htp1782k1",  
            host="localhost",  
            port="5432"       
        )
        conn.autocommit = True
        cursor = conn.cursor()

        # Kiểm tra database đã tồn tại chưa
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = 'token';")
        exists = cursor.fetchone()

        if not exists:
            cursor.execute("CREATE DATABASE token;")
            print("Database 'token' đã được tạo.")
        else:
            print("Database 'token' đã tồn tại.")

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Lỗi khi tạo database: {e}")

create_database()


Database 'token' đã tồn tại.


# 2. Check

In [5]:
def drop_tables_if_exist():
    try:
        conn = create_connection()
        if conn:
            cursor = conn.cursor()
            cursor.execute("DROP TABLE IF EXISTS dim_token CASCADE;")
            cursor.execute("DROP TABLE IF EXISTS dim_token_info CASCADE;")
            cursor.execute("DROP TABLE IF EXISTS fact_token_prices CASCADE;")
            conn.commit()
            print("Các bảng đã được xoá nếu chúng tồn tại.")
            cursor.close()
            conn.close()
        else:
            print("Không thể kết nối đến database để xoá bảng.")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xoá bảng: {e}")

# Chạy hàm xoá bảng
drop_tables_if_exist()


Kết nối đến PostgreSQL (token) thành công!
Các bảng đã được xoá nếu chúng tồn tại.


# 3 Create table

In [6]:
def setup_database():
    try:
        conn = create_connection()
        if conn:
            cursor = conn.cursor()
            with open(r'D:\HTP\Project-personal\CoinAnalytics\Main\Database\Setup_postgres.sql', 'r', encoding='utf-8') as file:
                sql_queries = file.read()
            cursor.execute(sql_queries)
            conn.commit()
            print("Database và tables đã được setup thành công!")
            cursor.close()
            conn.close()
        else:
            print("Không thể kết nối đến database để thực hiện setup.")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi setup database: {e}")

setup_database()


Kết nối đến PostgreSQL (token) thành công!
Database và tables đã được setup thành công!


# 4. Load postgres

In [7]:
def insert_data_from_csv():
    try:
        conn = create_connection()
        if conn:
            cursor = conn.cursor()

            # Chèn dữ liệu vào bảng dim_token
            tokens_df = pd.read_csv(r'D:\HTP\Project-personal\CoinAnalytics\Main\data\warehouse\dim_token.csv')
            for index, row in tokens_df.iterrows():
                cursor.execute("INSERT INTO dim_token (id, token_name, token) VALUES (%s, %s, %s)", 
                               (index + 1, row['token_name'], row['token']))
            
            # Chèn dữ liệu vào bảng dim_token_info
            token_info_df = pd.read_csv(r'D:\HTP\Project-personal\CoinAnalytics\Main\data\warehouse\dim_token_info.csv')
            for _, row in token_info_df.iterrows():
                cursor.execute(""" 
                    INSERT INTO dim_token_info (token_id, description, website, technical_doc, twitter, reddit, logo) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                """, (row['token_id'], row['description'], row['website'], row['technical_doc'], row['twitter'], row['reddit'], row['logo']))
            
            # Chèn dữ liệu vào bảng fact_token_prices
            token_prices_df = pd.read_csv(r'D:\HTP\Project-personal\CoinAnalytics\Main\data\warehouse\fact_token_prices.csv')
            for _, row in token_prices_df.iterrows():
                cursor.execute(""" 
                    INSERT INTO fact_token_prices (token_id, timestamp, open, high, low, close, volume, price_difference, 
                                                   price_percentage_change, mid_price, vwap) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (row['token_id'], row['timestamp'], row['open'], row['high'], row['low'], row['close'], row['volume'], 
                      row['price_difference'], row['price_percentage_change'], row['mid_price'], row['vwap']))

            conn.commit()
            print("Dữ liệu đã được chèn thành công vào các bảng!")
            cursor.close()
            conn.close()
        else:
            print("Không thể kết nối đến database để chèn dữ liệu.")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi chèn dữ liệu vào database: {e}")

# Chạy hàm chèn dữ liệu
insert_data_from_csv()


Kết nối đến PostgreSQL (token) thành công!
Dữ liệu đã được chèn thành công vào các bảng!


# Load MinIO

In [8]:
def check_and_create_bucket(bucket_name):
    try:
        # Kiểm tra nếu bucket đã tồn tại
        s3.head_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' đã tồn tại.")
    except ClientError as e:
        # Nếu bucket không tồn tại, tạo mới
        if e.response['Error']['Code'] == '404':
            print(f"Bucket '{bucket_name}' không tồn tại. Đang tạo mới...")
            s3.create_bucket(Bucket=bucket_name)
            print(f"Bucket '{bucket_name}' đã được tạo thành công ... Done")
        else:
            print(f"Error checking bucket: {e}")

# Kiểm tra và tạo bucket MinIO
check_and_create_bucket(bucket_name)


Bucket 'rawdata' đã tồn tại.


In [ ]:
def upload_files_to_minio(bucket_name, folder_path):
    try:
        # Lấy danh sách tất cả các file trong thư mục
        files = os.listdir(folder_path)
        
        for file in files:
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                print(f"Đang tải lên file {file}...")
                s3.upload_file(file_path, bucket_name, file)
                print(f"File {file} đã được tải lên ... Done")
            else:
                print(f"{file} không phải là file hợp lệ.")
    except Exception as e:
        print(f"Error uploading files: {e}")

# Chạy hàm tải file lên MinIO
upload_files_to_minio(bucket_name, r'D:\HTP\Project-personal\CoinAnalytics\Main\data\raw')
